In [1]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

# Replace these with your own client ID and client secret
CLIENT_ID = '4788c668b1514918954e31dfd7bd366d'
CLIENT_SECRET = 'c2c7f9f1133441db94b3afa3ef8832fe'
REDIRECT_URI = 'http://localhost:8888/callback'  

# Define the required scopes (permissions)
SCOPE = 'user-library-read playlist-read-private'

# Authentication
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=CLIENT_ID,
                                               client_secret=CLIENT_SECRET,
                                               redirect_uri=REDIRECT_URI,
                                               scope=SCOPE))

In [2]:
# Function to get recommendations based on a single seed track
def get_recommendations(seed_track_id, num_recommendations=10):
    recommendations = sp.recommendations(seed_tracks=[seed_track_id], limit=num_recommendations)
    recommended_tracks = []
    for track in recommendations['tracks']:
        recommended_tracks.append({
            'name': track['name'],
            'artist': track['artists'][0]['name'],
            'id': track['id']
        })
    return recommended_tracks

In [3]:
# Example usage: Replace with an actual track ID
seed_track_id = '3n3Ppam7vgaVa1iaRUc9Lp'  # Seed track ID
recommendations = get_recommendations(seed_track_id)
for track in recommendations:
    print(f"{track['name']} by {track['artist']}")

"The Take Over, The Breaks Over" by Fall Out Boy
Soul to Squeeze by Red Hot Chili Peppers
What I've Done by Linkin Park
Under Pressure - Remastered 2011 by Queen
1979 - Remastered 2012 by The Smashing Pumpkins
C'est La Vie by Stereophonics
Welcome to the Black Parade by My Chemical Romance
In The Heat Of The Moment by Noel Gallagher's High Flying Birds
Go Your Own Way - 2004 Remaster by Fleetwood Mac
Supersonic by Oasis


In [4]:
# Function to search for a track and get its ID
def search_track(track_name):
    results = sp.search(q=track_name, type='track', limit=1)
    track_id = results['tracks']['items'][0]['id']
    return track_id

In [6]:
# Example usage: Get recommendations based on user input
seed_track_id = search_track('Loser by Jaguar Twin')
recommendations = get_recommendations(seed_track_id)
for track in recommendations:
    print(f"{track['name']} by {track['artist']}")

I'm Okay by Honest Men
Dance in the Dark by Au/Ra
Long Time Coming - Live in a Living Room by Jagwar Twin
Summertime High by Half the Animal
KMS by Sub Urban
King of the Clouds by Panic! At The Disco
Somebody Better by The Million
fever dream by mxmtoon
Only Human by Ryan Mack
Teenager In Love by Neon Trees


: 

In [ ]:
import numpy as np
from sklearn.metrics import pairwise_distances

def find_similar_tracks(track_id, bp_track, scaler, clustering_model, clustered_data):
    # Step 1: Extract the track with the given track_id
    track = bp_track[bp_track['track_id'] == track_id]
    
    if track.empty:
        return "Track not found."
    
    # Step 2: Preprocess the track's features (use the same features you used in clustering)
    track_features = track[['BPM', 'Key']]  # Select the features you used for clustering
    track_features_scaled = scaler.transform(track_features)  # Apply the same scaling
    
    # Step 3: Predict the cluster of the track
    track_cluster = clustering_model.fit_predict(track_features_scaled)[0]
    
    # Step 4: Filter all tracks from the same cluster
    similar_tracks = clustered_data[clustered_data['Cluster'] == track_cluster]
    
    # Step 5: Calculate the distances between the track and all other tracks in the same cluster
    distances = pairwise_distances(track_features_scaled, similar_tracks[['BPM', 'Key']])
    
    # Add distances to the similar_tracks DataFrame and sort by closest distance
    similar_tracks['Distance'] = distances[0]
    similar_tracks_sorted = similar_tracks.sort_values(by='Distance')
    
    # Return the top 5 most similar tracks
    return similar_tracks_sorted.head(5)

# Example usage:
track_id_to_search = 4993057  # Replace with the track_id you're searching for
similar_tracks = find_similar_tracks(track_id_to_search, bp_track, scaler, agglo, clustered_data)
print(similar_tracks)
